This is a notebook that I have created to test the various features of the BaconBits cape.

I received the PocketBeagle when I enrolled in the [e-ale course](https://e-ale.org/) at the 2018 [Embedded Linux Conference](https://events.linuxfoundation.org/events/elc-openiot-north-america-2018/).

The resources for this course at contained on [github](https://github.com/e-ale).

The code that I am using to initially test the features of the BaconBits cape is located [here](https://github.com/e-ale/Code/tree/master/RESOURCES/pocketbeagle).

## Wifi setup

In order to easily use this notebook, I first need to get the PocketBeagle hooked up to the internet.

To do this, on my laptop, I type:
```
sudo su
iptables --table nat --append POSTROUTING --out-interface wlp3s0 -j MASQUERADE
iptables --append FORWARD --in-interface enx38d269566e5d -j ACCEPT
echo 1 > /proc/sys/net/ipv4/ip_forward
exit
```
and then on the PocketBeagle I type:
```
sudo su
ip route add default via 192.168.7.1
echo "nameserver 8.8.8.8" >> /etc/resolv.conf
exit
```

I would like to not have to run the above commands after every reboot of the PocketBeagle, but at the moment, I don't know had to do that.

I then cloned the git repository using:
```
git clone https://github.com/b-trav/BaconBitsIntro.git
```

Initially when I set up git I did these commands to set my username, email, and to allow me to push without typing my github password:
```
git config --global user.name "Ben Travaglione"
git config --global user.email "ben@travaglione.com"
git config --global credential.helper store
```

## Jupyter

Evidently, I am writing this notebook in Jupyter, and the notebook is designed to be run on the PocketBeagle, thus you need to get Jupyter up and running on the PocketBeagle.

This is accomplished with the following commands:

```
jupyter notebook --generate-config
```
I then allow external access to the notebook server by editing the jupyter config file:
```
vim .jupyter/jupyter_notebook_config.py
```
and uncommenting
```
c.NotebookApp.ip = '*'
```
and
```
c.NotebookApp.allow_root = True
```

I also need to add password access to the notebook server:
```
jupyter notebook password
```
I allow jupyter to boot as root at startup by adding:
```
@reboot /usr/bin/python /usr/local/bin/jupyter-notebook --config /home/debian/.jupyter/jupyter_notebook_config.py
```
to my crontab by typing
```
sudo crontab -e
```

These days I find Jupyter annoying to use without installing extensions, so these are the commands to install some Jupyter extensions:
```
sudo -H pip install --upgrade pip
sudo apt-get install python-dev libxml2-dev libxslt1-dev zlib1g-dev
sudo -H pip install jupyter_contrib_nbextensions
jupyter contrib nbextension install --user
jupyter nbextension enable codefolding/main
jupyter nbextension enable freeze/main
jupyter nbextension enable spellchecker/main
jupyter nbextension enable toggle_all_line_numbers/main
jupyter nbextension enable collapsible_headings/main
jupyter nbextension enable init_cell/main
jupyter nbextension enable scratchpad/main
jupyter nbextension enable toc2/main

sudo -H pip install jupyter_nbextensions_configurator
jupyter nbextensions_configurator enable --user
```

## Potentiometer

Let's look at the potentiometer on the BaconBits.

In bash, we can grab the value of the potentiometer very easily:

In [1]:
f = '/sys/bus/iio/devices/iio\:device0/in_voltage0_raw'
!ls $f
!cat $f


/sys/bus/iio/devices/iio:device0/in_voltage0_raw
465


This can be a bit trickier in python because it does not seem to like the filename for some reason:

In [2]:
with open(f,'r') as pot:
    val = pot.readline()
    print(val)

IOError: [Errno 2] No such file or directory: '/sys/bus/iio/devices/iio\\:device0/in_voltage0_raw'

I can get around this problem by creating a symbolic link:

In [3]:
!rm potentiometer
!ln -s /sys/bus/iio/devices/iio\:device0/in_voltage0_raw potentiometer

In [4]:
lnf = 'potentiometer'
with open(lnf,'r') as pot:
    val = pot.readline()
    print(val)

465



But this seems like a bit of a hack!

# User Button

In bash, we can get the status of the user button with:

In [5]:
!cat /sys/class/gpio/gpio45/value

1


In [10]:
from IPython.display import clear_output

button_filename = '/sys/class/gpio/gpio45/value'
with open(button_filename,'r') as button_file:
    val = button_file.readline()
last_val = val
print("Current User-button value: {} (Interrupt the kernel to quit)".format(val))
while True:
    with open(button_filename,'r') as button_file:
        val = button_file.readline()
    if not (val == last_val):
        last_val = val
        clear_output()
        print("Current User-button value: {} (Interrupt the kernel to quit)".format(val))
        


Current User-button value: 1
 (Interrupt the kernel to quit)


KeyboardInterrupt: 

# SPI 7-segment LEDs

In bash, we can turn on the 7-segment LEDs buy typing:
```
echo -ne '\x40\x01\x02' | sudo tee /dev/spidev2.1 | xxd
```
In python, we can do the following:

In [23]:
import binascii
hexstr = '400102'
binstr = binascii.unhexlify(hexstr)
print(binstr)

@


In [25]:
with open('/dev/spidev2.1','wb') as f:
    f.write(binstr)